In [1]:
from layers import *

In [2]:
lora_layer = LoRALayer(10, 0.01, 0.1, True)
assert lora_layer.r == 10
assert lora_layer.lora_alpha == 0.01
assert isinstance(lora_layer.lora_dropout, Dropout)
assert lora_layer.merged == False
assert lora_layer.merge_weights == True

In [3]:
# Initialize the model
embedding = Embedding(100, 50, r=10, lora_alpha=0.01, merge_weights=True)
x = tf.random.uniform(shape=(32, 10), minval=0, maxval=100, dtype=tf.int32)
optimizer = tf.keras.optimizers.Adam()

2023-07-04 10:58:11.015507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-04 10:58:11.025054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-04 10:58:11.025679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-04 10:58:11.026872: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
# Function for the forward pass
def forward_pass(x):
    return embedding(x)

# Function for a single training step
def train_step(inputs):
    with tf.GradientTape() as tape:
        predictions = forward_pass(inputs)
        loss = tf.reduce_mean(predictions)  # Dummy loss

    gradients = tape.gradient(loss, embedding.trainable_variables)
    optimizer.apply_gradients(zip(gradients, embedding.trainable_variables))
    return loss

# Function for a single evaluation step
def eval_step(inputs):
    predictions = forward_pass(inputs)
    return tf.reduce_mean(predictions)  # Dummy score

In [5]:
# Test in training mode with LoRA enabled (r > 0)
assert not embedding.merged
embedding.train(True)
assert embedding.trainable
assert not embedding.merged  # This line was modified
print("Shape of lora_A:", embedding.lora_A.shape)
print("Shape of lora_B:", embedding.lora_B.shape)
print("Shape of embeddings:", embedding.embeddings.shape)

loss = train_step(x)
assert loss is not None

Shape of lora_A: (100, 10)
Shape of lora_B: (10, 50)
Shape of embeddings: (100, 50)


ValueError: Exception encountered when calling layer "embedding" (type Embedding).

Attempt to convert a value (Ellipsis) with an unsupported type (<class 'ellipsis'>) to a Tensor.

Call arguments received:
  • x=tf.Tensor(shape=(32, 10), dtype=int32)

In [6]:
# Test resetting parameters
embedding.reset_parameters()
assert tf.reduce_all(embedding.lora_A == 0)
assert tf.reduce_all(embedding.lora_B == 0)

NameError: name 'embedding' is not defined

In [8]:
# Test in evaluation mode with LoRA enabled (r > 0) and weights merged
embedding.train(False)
assert not embedding.trainable
assert embedding.merged
score = eval_step(x)
assert score is not None


AssertionError: 